In [10]:
import numpy as np
from scipy.io.wavfile import read
import matplotlib.pyplot as plt

# 读取wav文件，得到采样率和数据

# sample_rate, data = read('C:\\Users\\88486\\Desktop\\WorkSpace\\Python\\通信原理\\suno.wav') #windows路径

sample_rate, data = read('suno.wav') #linux路径
print(sample_rate,data)
# 创建载波信号
time = np.arange(len(data))/float(sample_rate)#载波时间
carrier_freq=sample_rate*2#载波频率


48000 [[-30 -37]
 [-42 -50]
 [-35 -42]
 ...
 [-31 -40]
 [-37 -46]
 [-30 -37]]


In [11]:
carrier = np.cos(2.0 * np.pi * carrier_freq * time)#产生载波信号
modulated_data = np.empty_like(data)
modulated_data[:, 0] = carrier * data[:, 0]#左声道调制
modulated_data[:, 1] = carrier * data[:, 1]#右声道调制


In [28]:
# 接下来对AM信号进行解调
# 生成解调信号
demodulated_data = np.empty_like(modulated_data)
demodulated_data[:, 0] = modulated_data[:, 0] * carrier#左声道解调
demodulated_data[:, 1] = modulated_data[:, 1] * carrier#右声道解调
# plt.figure()
# plt.plot(demodulated_data)
# if(demodulated_data==data).all():
#     print("解调成功")
# else:
#     print("解调失败")
cha=demodulated_data-data
print("max(cha[:,0]):{}".format(max(cha[:,0])))
print(f"max(cha[:,1]):{max(cha[:,1])}")
print(f"max(data[:,0]):{max(data[:,0])}")
print(f"max(data[:,1]):{max(data[:,1])}")
print(f"max(demodulated_data[:,0]):{max(demodulated_data[:,0])}")
print(f"max(demodulated_data[:,1]):{max(demodulated_data[:,1])}")
if(max(cha[0])//max(data[:,0])<0.001):
    print("左声道解调成功")
else:
    print("左声道解调失败")

if(max(cha[1])//max(data[:,1])<0.001):
    print("右声道解调成功")
else:
    print("右声道解调失败")

# for i in range(0,len(data)):
# print(cha)
# sum_left=sum_right=0
# for i in range(0,len(cha)):
#     if cha[i][0]!=0:
#         sum_left+=1
#     if cha[i][1]!=0:
#         sum_right+=1
# print(f"sum_left:{sum_left}")
# print(f"sum_right:{sum_right}")

# if(demodulated_data[:,0]==data[:,0]).all():
#     print("左声道解调成功")
# else:
#     print("左声道解调失败")
# if(demodulated_data[:,1]==data[:,1]).all():
#     print("右声道解调成功")
# else:
#     print("右声道解调失败")

max(cha[:,0]):2
max(cha[:,1]):2
max(data[:,0]):26626
max(data[:,1]):27485
max(demodulated_data[:,0]):26626
max(demodulated_data[:,1]):27485
左声道解调成功
右声道解调成功


In [17]:
print(len(data))

4752000


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 定义参数
sample_rate = 1000 # 采样率
carrier_freq = 50 # 载波频率
time = np.arange(0, 1, 1/sample_rate) # 时间数组

# 生成载波信号
carrier = np.cos(2 * np.pi * carrier_freq * time)

# 绘制载波信号
plt.figure()
plt.plot(time, carrier)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Carrier signal')
plt.grid(True)
plt.show()